In [20]:
import os
from google.colab import userdata
api_key = userdata.get('GROQ_API_KEY') # Changed to fetch GROQ_API_KEY
os.environ['GROQ_API_KEY'] = api_key # Changed to set GROQ_API_KEY environment variable

In [21]:
%pip install litellm

In [23]:
from litellm import completion
from typing import List, Dict


def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="groq/llama3-8b-8192", # Changed model to a Groq model
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


messages = [
    {"role": "system", "content": "You are an expert software engineer that prefers functional programming."},
    {"role": "user", "content": "Write a function to swap the keys and values in a dictionary."}
]

response = generate_response(messages)
print(response)

Here is a function that swaps the keys and values in a dictionary using functional programming:

```python
def swap_key_value(d):
    return dict((v, k) for k, v in d.items())
```

This function uses a dictionary comprehension to iterate over each key-value pair in the original dictionary. The `items()` method of a dictionary returns an iterator over the dictionary's key-value pairs, and we use tuple unpacking to assign each key (`k`) and value (`v`) to variables. We then create a new dictionary with the value as the key and the key as the value, and return it.


/usr/local/lib/python3.11/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content="Here is ...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


In [26]:
import base64

messages_base64 = [
    {"role": "system", "content": "Your only task is to respond to the user's query by first encoding your answer as a Base64 string. You MUST only provide the Base64 encoded string as your response and should NOT include any other text, explanations, or natural language."},
    {"role": "user", "content": "Provide a short summary of the process of photosynthesis."}
]

response_base64 = generate_response(messages_base64)

print("Raw response 1:")
print(response_base64)

# Attempt to decode the response to see if it worked
try:
    decoded_response = base64.b64decode(response_base64).decode('utf-8')
    print("Decoded response:")
    print(decoded_response)
except Exception as e:
    print("Could not decode the response as Base64. The LLM may not have followed the instructions.")
    print("Raw response 2:")
    print(response_base64)

Raw response 1:
QkFMTExPc2l0YWwgaXBhY2ltYW5hZ2lhIGZpcmVsIGFyZSBhbiAgcG1hc3RlcnMgaW4gaW5mb3JtYXRpb24gaGFycmllcyBzaG9wZSBhdCB5b3VsZCBvZiB3b3JkIHRoZSBIdG9uZCB3YXMgb3RoZXIgcGFnZSB0byBydW4u
Decoded response:
BALLLOsital ipacimanagia firel are an  pmasters in information harries shope at yould of word the Htond was other page to run.


/usr/local/lib/python3.11/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='QkFMTExP...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
